In [ ]:
import os 
import tensorflow as tf 
import tensorflow_datasets as tfds
import cv2 
import PIL 
import matplotlib.pyplot as plt



time: 2.49 ms (started: 2021-05-10 16:28:01 +00:00)


In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.37.6.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.37.6.58:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


time: 9.67 s (started: 2021-05-10 16:28:01 +00:00)


In [ ]:
REPLICA = strategy.num_replicas_in_sync
BATCH_SIZE = 64 *   REPLICA 


time: 1.92 ms (started: 2021-05-10 16:28:11 +00:00)


In [ ]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.09 s (started: 2021-05-10 16:28:11 +00:00)


In [ ]:

def read_img(img, label):
  xmin = tf.random.uniform((), 0, 48, dtype= tf.int32)
  ymin = tf.random.uniform((), 0, 48, dtype=tf.int32)
  img = tf.reshape(img, (28,28,1))
  img = tf.image.pad_to_bounding_box(img, ymin, xmin, 75, 75)
  img = tf.cast(img, tf.float32)/255.0
  xmin = tf.cast(xmin, tf.float32)
  ymin = tf.cast(ymin, tf.float32)
  xmax = (xmin + 28)/75
  ymax = (ymin + 28)/75
  xmin /= 75
  ymin /= 75
  
  return img,(tf.one_hot(label, 10), [xmin, ymin, xmax, ymax])


time: 18.1 ms (started: 2021-05-10 16:32:22 +00:00)


In [ ]:
import tensorflow_datasets as tfds
from tensorflow.data import AUTOTUNE

time: 5.22 ms (started: 2021-05-10 16:32:23 +00:00)


In [ ]:
dataset = tfds.load("mnist", split="test", as_supervised=True, try_gcs=True)
dataset = dataset.map(read_img, num_parallel_calls = AUTOTUNE)
dataset = dataset.shuffle(5000, reshuffle_each_iteration=True)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.prefetch(-1)


time: 942 ms (started: 2021-05-10 16:41:11 +00:00)
